## Ricavare vari indici statistici e di network science dai dati ricavati col data gathering e costruire le community

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import networkx.algorithms.community as community
import scipy.stats as sp
import pickle
#con questo codice creo un file txt in cui, per ogni giorno di data gathering, sono riportati alcuni indici utili ai fini del progetto, e inoltre costruisco le community usando l'algoritmo di louvain

#qui decido se fare analisi dei dati delle classifiche viral o global
#with open('viral.txt', 'w') as file:
with open('viral.txt', 'w') as file:   

    #itero sui 15 giorni di data gathering
    for z in range (1,16,14):
        #with open(f'global_genres0{z}.pkl', 'rb') as f:
        #    grafo = pickle.load(f)
        #with open(f'global{z}.pkl', 'rb') as f:
        #    grafo = pickle.load(f)
        #apro il grafo relativo al giorno in cui ho fatto data gathering
        with open(f'viral{z}.pkl', 'rb') as f:
            grafo = pickle.load(f)
            print(nx.density(grafo))
                
        '''
        with open(f'viral{z}.pkl', 'rb') as f:
            grafo = pickle.load(f)
        '''        
        #estraggo i gradi dei nodi, il numero di archi, il grado medio pesato e non pesato, la mediana dei gradi, la moda dei gradi e calcolo la densità della rete
        degree = dict(grafo.degree(weight = 'weight'))
        campione_grado = list(degree.values())
        file.write(f'--DATA GATHERING 1{z} APRILE \n')
        file.write(f'Numero di archi: {grafo.size()} \n')
        file.write(f'Grado medio non pesato: {(grafo.size()*2)/grafo.order()} \n')
        degree = dict(grafo.degree(weight = 'weight'))
        file.write(f'Grado medio pesato: {np.mean(list(degree.values()))} \n')
        file.write(f'Mediana: {np.median(campione_grado)} \n')
        file.write(f'Moda: {sp.mode(campione_grado)} \n')
        file.write(f'Densità: {(2*grafo.size())/(grafo.order()*(grafo.order()-1))} \n')

        #prendo il grado massimo
        max_degree = max(degree.values())
        max_country = ''
        #e cerco il paese corrispondente al grado massimo
        for k,v in grafo.degree(weight = 'weight'):
            if v == max_degree:
                max_country = k
                break
        file.write(f'Nodo grado massimo: {grafo.nodes.data("label")[max_country]} valore: {max_degree} \n')

        #stessa cosa per il grado minimo
        min_degree = min(degree.values())
        min_country = ''
        for k,v in grafo.degree(weight = 'weight'):
            if v == min_degree:
                min_country = k
                break
        file.write(f'Nodo grado minimo: {grafo.nodes.data("label")[min_country]} valore: {min_degree} \n')

        #ricavo l'arco di peso massimo tra due nodi del grafo
        edges = list(grafo.edges.data("weight"))
        max_archi = (sorted(edges, key=lambda x:x[2], reverse=True))[:5]
        for i,j,k in max_archi:
            file.write(f'Arco peso maggiore tra: {grafo.nodes.data("label")[i]} e {grafo.nodes.data("label")[j]} con valore: {k} \n')

        #e quello di peso minimo
        min_archi = (sorted(edges, key=lambda x:x[2], reverse=True))[len(edges)-5:]
        for i,j,k in min_archi:
            file.write(f'Arco peso minore tra: {grafo.nodes.data("label")[i]} e {grafo.nodes.data("label")[j]} con valore: {k} \n')


        degree = dict(grafo.degree(weight = 'weight')) 
        campione_grado = list(degree.values())
        percentile_95 = np.percentile(campione_grado,95)
        #cerco i nodi hub considerando quelli che hanno grado maggiore del 95° percentile del grado dei nodi
        hub_nodi = [k for k,v in degree.items() if v>= percentile_95]
        file.write(f'Hub:')
        for k in hub_nodi:
            file.write(f' {grafo.nodes.data("label")[k]}')

        #vedo se il grafo è connesso
        file.write(f'\nIl grafo è connesso: {nx.is_connected(grafo)}')

        #calcolo la transitività
        file.write(f'\nTransitività: {nx.transitivity(grafo)}')
        #calcolo l'assortatività del grado dei nodi
        file.write(f'\nAssortatività grado nodi: {nx.degree_assortativity_coefficient(grafo)}')
        
        d = dict(grafo.nodes(data="label"))
        
        communities = nx.community.louvain_communities(grafo, seed = 67, resolution=1.1)
        for c in communities:
            print(c)
        print('\n\n')

        #ricavo le eigenvector centrality
        eigen_con_pesi = nx.eigenvector_centrality(grafo, weight='weight')
        max5_pesi = sorted(eigen_con_pesi, key=eigen_con_pesi.get, reverse=True)[:5]
        min5_pesi = sorted(eigen_con_pesi, key=eigen_con_pesi.get, reverse=True)[67:]
        #e cerco quelle minime e massime
        file.write(f'\nEigenvector massima:')
        for k in max5_pesi:
            file.write(f' {grafo.nodes.data("label")[k]}')
        file.write(f'\nEigenvector minima:')
        for k in min5_pesi:
            file.write(f' {grafo.nodes.data("label")[k]}')

        file.write('---------------------\n')

        #uso l'algoritmo di louvain per trovare le community
        l_coms = community.louvain_communities(grafo, seed=67, resolution = 1.1)
        community_louvain = {i:e for i, e in enumerate(l_coms)}
        for i, com in community_louvain.items():
            for n in com:
                grafo.nodes[n]['com'] = i 
        #e salvo le communjty in un file gexf
        #nx.write_gexf(grafo, f'viral_community0{z}.gexf')
        nx.write_gexf(grafo, f'global_community0{z}.gexf')
        #z = z + 14

EOFError: Ran out of input

## Determinare i generi più e meno presenti nelle classifiche top e viral

In [ ]:
import networkx as nx
import pickle

# Carica un grafo da un file .pkl
def load_graph_from_pickle(filename):
    with open(filename, 'rb') as f:
        graph = pickle.load(f)
    return graph

# Costruisce un dizionario delle occorrenze dei nodi nei diversi grafi
def build_node_count_dict():
    node_count = {}
    for i in range(1, 16):
        #filename = f"viral_genres0{i}.pkl"
        filename = f"global_genres0{i}.pkl"

        graph = load_graph_from_pickle(filename)
        nodes = list(graph.nodes())
        for node in nodes:
            node_count[node] = node_count.get(node, 0) + 1
    return node_count

# Esempio di utilizzo
if __name__ == "__main__":
    # Costruisce il dizionario delle occorrenze dei nodi
    node_count = build_node_count_dict()
    
    # Ordina il dizionario per valore (occorrenze)
    sorted_node_count = dict(sorted(node_count.items(), key=lambda item: item[1], reverse=True))
    
    # Stampa il dizionario ordinato
    print("Dizionario delle occorrenze dei nodi (ordinato per valore):")
    for node, count in sorted_node_count.items():
        print(node, ":", count)


## Creazione del grafico ECCDF per la distribuzione del grado nei vari giorni

In [ ]:

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from statsmodels.distributions.empirical_distribution import ECDF
import pickle
import os
from matplotlib import font_manager as fm, rcParams
import matplotlib as mpl

#apro tutti i file pkl corrispondenti ai grafi costruiti col data gathering
for z in range (1,16):
    giorno = 11
    with open(f'global{z}.pkl', 'rb') as f:
    #with open(f'viral{z}.pkl', 'rb') as f:
        grafo = pickle.load(f)
        degree = dict(grafo.degree(weight = 'weight'))
        campione_degree = list(degree.values())

        min_t = min(campione_degree)
        max_t = max(campione_degree)

        count_rete, bins_rete = np.histogram(campione_degree, bins = np.arange(min_t,max_t+2))

        pdf_rete = count_rete / grafo.order()

        cdf_fb = ECDF(campione_degree)
        x = np.unique(campione_degree)
        y = cdf_fb(x)

        # Visualizzazione
        fig_cdf_function = plt.figure(figsize=(21,9))
        fig_cdf_function.set_facecolor('#1c1e21')
        assi = fig_cdf_function.gca()
        assi.set_facecolor('#1c1e21')

        fpath = os.path.join(mpl.get_data_path(), "fonts/ttf/DejaVuSerif.ttf")
        prop = fm.FontProperties(fname=fpath)
        fname = os.path.split(fpath)[1]

        assi.set_xlabel('X-axis')
        assi.set_ylabel('Y-axis')

        assi.spines['left'].set_color('white')  
        assi.spines['right'].set_color('white')         # setting up Y-axis tick color to red
        assi.spines['top'].set_color('white')
        assi.spines['bottom'].set_color('white')   
        assi.tick_params(colors='white', which='both')

        assi.loglog(x,1-y,color = '#00d95a', linestyle = ':', marker= 'o',ms = 10)
        plt.title(f'Top {giorno+z} aprile '.format(fname), fontproperties=prop, size = 30, color='#00d95a')
        #plt.title(f'Viral {giorno+z} aprile'.format(fname), fontproperties=prop, size = 30, color='#00d95a')
        assi.set_xlabel('Grado'.format(fname), fontproperties=prop, size = 30, color='#00d95a')
        assi.set_ylabel('ECCDF'.format(fname), fontproperties=prop, size = 30, color='#00d95a')

        #Salvo il grafico in un png in modo tale da mostrarlo nel powerpoint della presentazione
        plt.savefig(f'global_ECDF{z}.png')
        #plt.savefig(f'viral_ECDF{z}.png')

## Contare la frequenza delle canzoni nei vari giorni di data gathering

In [ ]:
import csv
import glob

def count_titles_in_csv_files():
    # Dizionario per memorizzare il conteggio dei titoli
    title_count = {}

    # Cerca tutti i file CSV nella stessa cartella del sorgente Python che iniziano con "z-global.csv"
    for i in range(1,16):
        #with open(f'0{i}-global.csv', 'r', encoding = 'utf-16') as csvfile:
        with open(f'0{i}-viral.csv', 'r', encoding = 'utf-16') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                # Ottiene il titolo dalla riga
                title = row[2:]
                # Aggiorna il conteggio del titolo nel dizionario
                title_count[title[0]] = title_count.get(title[0], 0) + 1

    return title_count

# Ottieni il conteggio dei titoli nei file CSV
titles_count = count_titles_in_csv_files()

sorted_titles_count = dict(sorted(titles_count.items(), key=lambda item: item[1], reverse=True))

# Stampare il conteggio dei titoli
for title, count in sorted_titles_count.items():
    print(f"{title}: {count}")
